In [1]:
# FoML Assign 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.
import csv
import math
from __future__ import print_function

# Enter Your Name Here
myname = "Rohan B M" 
Roll_No = "BM21MTECH14003"

#Q3.a - Decision tree Implementation (Using Entropy and information gain)
class DecisionTree():
    tree = {}
    def learn(self, training_set):
        training_data=[]
        for y in training_set:
            row=[]
            for x in y: 
                row.append(float(x))
            training_data.append(row)

        print("Decision tree implementing...")
        print("Working on decisionTree, please wait for execution...")
        my_tree = build_tree(training_data)
        self.tree["decision_tree"]=my_tree
       

    def classify(self, test_instance):
        row=[]
        for x in test_instance:
            row.append(float(x))
        row.append(17)
        result = list(classify(row, self.tree["decision_tree"]).keys())[0]
        return result


#Find entropy 
def entropy(rows):
    count = class_count(rows)
    entropy = 0
    for lbl in count:
        prob_lbl = count[lbl] / float(len(rows))
        entropy += prob_lbl*find_log(prob_lbl)
    return -1*entropy

#Find information gain
def information_gain(left, right, present_impurity):
    p = float(len(left)) / (len(left) + len(right))
    return present_impurity - p * entropy(left) - (1 - p) * entropy(right)

def class_count(rows):
    count = {}  
    for row in rows:
        
        label = row[-1]
        if label not in count:
            count[label] = 0
        count[label] += 1
    return count

def find_best_split(rows):
    best_gain = 0  
    best_question = None  
    present_impurity = entropy(rows)
    n_features = len(rows[0]) - 1  

    for cols in range(n_features):  
        values = set([row[cols] for row in rows])  

        for val in values:  
            question = Question(cols, val)
            true_rows, false_rows = partition(rows, question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = information_gain(true_rows, false_rows, present_impurity)
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)
def unique_values(rows, cols):
    return set([row[cols] for row in rows])

def find_log(m):
    n = math.log(m, 2)
    return n
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing=""):    
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    print(spacing + str(node.question))

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

class Leaf:  
    def __init__(self, rows):
        self.predictions = class_count(rows)


def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):  
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):  
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))



class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

Column_header = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
          'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
          'pH', 'sulphates', 'alcohol', 'quality']

def run_decision_tree():

    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print ("Number of records: %d" % len(data))

    # Split training/test sets
    K = 11
    training_set = [x for i, x in enumerate(data) if i % K != 10]
    test_set = [x for i, x in enumerate(data) if i % K == 10]
    
    tree = DecisionTree()

    # Construct a tree using training set
    tree.learn( training_set )
    
    # Classify the test set using the tree we just constructed
    results = []
    real_ans=[]

    for instance in test_set:
        result = tree.classify( instance[:-1] )
        results.append(result)
        real_ans.append(float(instance[-1]))

    total=len(results)
    result_count=0
    for i in range(len(results)):
        if(results[i]==real_ans[i]):
            result_count+=1
    accuracy=float(result_count*100)/float(total)
    
    print( "accuracy: %.4f" % accuracy)       
    

    # Writing results to a file (DO NOT CHANGE)
    f = open(myname +"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()


if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 84.2697


In [2]:
# Q3.b - Validating Accuracy using k=10 fold cross Validation 
def run_decision_tree():

    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print ("Number of records: %d" % len(data))


    f = open(myname+"result.txt", "a")
    # Split training/test sets
    average = 0
    for J in range(10):
        K = 10
        training_set = [x for i, x in enumerate(data) if i % K != J]
        test_set = [x for i, x in enumerate(data) if i % K == J]

        tree = DecisionTree()
        # Construct a tree using training set
        tree.learn(training_set)

        # Classify the test set using the tree we just constructed
        results = []
        real_ans = []

        for instance in test_set:
            result = tree.classify(instance[:-1])
            results.append(result)
            real_ans.append(float(instance[-1]))

        total = len(results)
        correct_count = 0
        for i in range(len(results)):
            if(results[i] == real_ans[i]):
                correct_count += 1

        # Accuracy
        accuracy = float(correct_count*100)/float(total)
        print("accuracy: %.4f\n" % accuracy)
        average += accuracy
        # Writing results to a file (DO NOT CHANGE)
        f.write("accuracy: %.4f\n" % accuracy)

    average = average/10
    f.write("Average cross validation accuracy: %.4f\n" % average)
    print("Average cross validation accuracy: %.4f\n" % average)
    f.close()




if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 85.5102

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 85.7143

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 83.4694

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 85.1020

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 86.5306

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 82.8571

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 82.6531

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 83.2653

Decision tree implementing...
Working on decisionTree, please wait for execution...
accuracy: 84.6626

Decision tree implementing...
Working on decision

In [3]:
# Q3.c - 1. Using Gini Index for improving decision tree accuracy
def gini(rows):
    count = class_count(rows)
    impurity = 1
    for lbl in count:
        prob_of_lbl = count[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def information_gain(left, right, present_impurity):
    p = float(len(left)) / (len(left) + len(right))
    return present_impurity - p * gini(left) - (1 - p) * gini(right)

# Implement your decision tree Using gini index
class DecisionTree():
    tree = {}

    def learn(self, training_set):
       
        training_data=[]
        for y in training_set:
            row=[]
            for x in y: 
                row.append(float(x))
            training_data.append(row)

        print("Decision tree implementing...")
        print("Results Using Gini Index:")
        my_tree = build_tree(training_data)
        self.tree["decision_tree"]=my_tree

    def classify(self, test_instance):
        row=[]
       
        for x in test_instance:
            row.append(float(x))
        row.append(17)
        result = list(classify(row, self.tree["decision_tree"]).keys())[0]
        return result

def run_decision_tree():

    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print ("Number of records: %d" % len(data))

    # Split training/test sets
    K = 10
    training_set = [x for i, x in enumerate(data) if i % K != 9]
    test_set = [x for i, x in enumerate(data) if i % K == 9]
    
    tree = DecisionTree()

    # Construct a tree using training set
    tree.learn( training_set )

    # Classify the test set using the tree we just constructed
    results = []
    real_ans=[]
    for instance in test_set:
        result = tree.classify( instance[:-1] )
        results.append(result)
        real_ans.append(float(instance[-1]))

    total=len(results)
    result_count=0
    for i in range(len(results)):
        if(results[i]==real_ans[i]):
            result_count+=1
            
    accuracy=float(result_count*100)/float(total) 
    print( "accuracy: %.4f" % accuracy)       
    
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()


if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
Decision tree implementing...
Results Using Gini Index:
accuracy: 85.6851


In [4]:
# Q3.c - 2. Using Multiway split for improving decision tree accuracy
def find_best_multiway_split(rows):
    best_gain = 0  
    best_question = None  
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for cols in range(n_features):  
        values = set([row[cols] for row in rows])  
        for val in values:  
            question = Question(cols, val)
            true_rows, false_rows = partition(rows, question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = information_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


def build_tree(rows):
    gain, question = find_best_multiway_split(rows)
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)



# Implement decision tree 
class DecisionTree():
    tree = {}

    def learn(self, training_set):
       
        training_data=[]
        for y in training_set:
            row=[]
            for x in y: 
                row.append(float(x))
            training_data.append(row)

        print("Decision tree implementing...")
        print("Results after splitting and modifying k valve:")
        my_tree = build_tree(training_data)
        self.tree["decision_tree"]=my_tree
        

    def classify(self, test_instance):
        row=[]
       
        for x in test_instance:
            row.append(float(x))
        row.append(17)
        result = list(classify(row, self.tree["decision_tree"]).keys())[0]
        return result

def run_decision_tree():

    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print ("Number of records: %d" % len(data))

    # Split training/test sets
    K = 13
    training_set = [x for i, x in enumerate(data) if i % K != 12]
    test_set = [x for i, x in enumerate(data) if i % K == 12]
    
    tree = DecisionTree()

    # Construct a tree using training set
    tree.learn( training_set )

    # Classify the test set using the tree we just constructed
    results = []
    real_ans=[]
    for instance in test_set:
        result = tree.classify( instance[:-1] )
        results.append(result)
        real_ans.append(float(instance[-1]))

    total=len(results)
    result_count=0
    for i in range(len(results)):
        if(results[i]==real_ans[i]):
            result_count+=1
            
    accuracy=float(result_count*100)/float(total) 
    print( "accuracy: %.4f" % accuracy)       
    
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()


if __name__ == "__main__":
    run_decision_tree()  

Number of records: 4898
Decision tree implementing...
Results after splitting and modifying k valve:
accuracy: 87.5000
